In [27]:
###########################
### MODULES NECESSAIRES ###
###########################


# Import the necessary libraries
import numpy as np
import pandas as pd
import math
import fonctionsSupervisedLearning as fsl

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn import svm

from auxFonctions import AminoAcid

In [28]:
# Read data from a file into a list of entries
with open('data/SIG_13.red', 'r') as file:
    entries = file.read().split('\n   ')



In [29]:
##############################
## RECUPERATION DES DONNÉES ##
##############################


# Process each entry
processed_entries = [fsl.process_entry(entry) for entry in entries]

# Create a DataFrame
df = pd.DataFrame(processed_entries)

# Get the position of the cleavage site
cleavage_site_position = df['Annotation'].apply(lambda x: x.find('C'))

# Split the primary structure into a list of amino acids
amino_acid_seq = df['Primary Structure'].apply(lambda x: list(x))

## Traitement des données

Put words in vector and vice versa

In [30]:
df_exploitable = fsl.convert_df_to_vectors(df)

In [31]:
n = 17  # length of the subsequence
svm_model_in = svm.SVC(kernel='rbf', C=1, random_state=42)

svm_model_pos = svm.SVC(kernel='linear', C=1, random_state=42)




Train

In [35]:
# svm_model_in, svm_model_pos, accuracy_in, accuracy_pos = fsl.create_model(12, df_exploitable, random_state=42, nb_letters = 26, kernel_in = 'rbf', kernel_pos = 'linear', C_in = 1, C_pos = 1)

# df_exploitable_100 = df_exploitable.iloc[:,1:100]

# C_list = [0.1,1,2]
# kernel_list = ['linear', 'rbf', 'sigmoid', 'poly']
# n_list = [7, 12, 17]
# results = []

# for C_in in C_list:
#     for C_pos in C_list:
#         for kernel_in in kernel_list:
#             for kernel_pos in kernel_list:
#                 for n in n_list:
#                     svm_model_in, svm_model_pos, accuracy_in, accuracy_pos = fsl.create_model(n, df_exploitable.iloc[:,1:100], random_state=42, nb_letters = 26, kernel_in = kernel_in, kernel_pos = kernel_pos, C_in = C_in, C_pos = C_pos)
#                     results.append((C_in, C_pos, kernel_in, kernel_pos, n, accuracy_in, accuracy_pos))

# results_df = pd.DataFrame(results, columns=['C_in', 'C_pos', 'kernel_in', 'kernel_pos', 'n', 'accuracy_in', 'accuracy_pos'])
# results_df.to_csv('results.csv', index=False)
X_train, X_test, pos_train, pos_test = fsl.test_train_split_random_pos(df_exploitable, n, random_state=42)
in_train = ~np.isnan(pos_train)
in_test = ~np.isnan(pos_test)



X_in_train = X_train[in_train==1]
pos_train = pos_train[~np.isnan(pos_train)]

from sklearn.model_selection import GridSearchCV

# Define the parameter grid to search
param_grid_in = {
    'C': [0.1, 1, 10, 100],  # Regularization parameter
    'gamma': ['scale', 'auto', 0.01, 0.1, 1],  # Kernel coefficient for 'rbf', 'poly' and 'sigmoid'
    'kernel': ['rbf', 'poly', 'sigmoid']  # Different types of kernels
}

# Initialize the classifier
svm_model_in = svm.SVC(random_state=42, class_weight='balanced')  # Using balanced class weights

# Setup the GridSearchCV
grid_search_in = GridSearchCV(svm_model_in, param_grid_in, cv=5, scoring='f1', verbose=2, n_jobs=-1)

# Assuming X_train and in_train have been defined as your features and target variable respectively
grid_search_in.fit(X_train, in_train)
best_in_model = grid_search_in.best_estimator_

print("Best parameters for inclusion model:", grid_search_in.best_params_)
print("Best score for inclusion model:", grid_search_in.best_score_)


Fitting 5 folds for each of 60 candidates, totalling 300 fits
Best parameters for inclusion model: {'C': 1, 'gamma': 'scale', 'kernel': 'sigmoid'}
Best score for inclusion model: 0.7055672306848366


In [34]:
svm_model_pos = svm.SVC(random_state=42, class_weight='balanced')  # Using balanced class weights
grid_search_pos = GridSearchCV(svm_model_pos, param_grid_in, cv=5, scoring='f1', verbose=2, n_jobs=-1)

grid_search_pos.fit(X_in_train, pos_train)
best_pos_model = grid_search_pos.best_estimator_

print("Best parameters for position model:", grid_search_pos.best_params_)
print("Best score for position model:", grid_search_pos.best_score_)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


c:\Users\rabas\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:952: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan]
  warnings.warn(


Best parameters for position model: {'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf'}
Best score for position model: nan


In [ ]:
svm_model_in, svm_model_pos, accuracy_in, accuracy_pos = fsl.create_model(17, df_exploitable, random_state=42, nb_letters = 26, kernel_in = best_results['kernel_in'], kernel_pos = best_results['kernel_pos'], C_in = best_results['C_in'], C_pos = best_results['C_pos'])

# svm_model_in.predict(fsl.word_to_vector('LTDACFI').tolist())

NameError: name 'fsl' is not defined

In [ ]:
# print(fsl.find_cleavage(test, threshold = 0.1))
pos_pred = np.array([fsl.find_cleavage(X,svm_model_in, svm_model_pos, threshold = 0.0, n= 17) for X in df_exploitable['P_Structure_vector'][:100]])

in_pred = 1-np.isnan(pos_pred)

pos_pred[in_pred == 0] = 0

print(accuracy_score(df_exploitable["Annotation_pos"][:100], pos_pred))

0.53
